# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from IPython.display import Image

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Get the cities saved in the previous jupyter notebook
cities_data = pd.read_csv("../output_data/cities.csv")
cities_data.dropna(inplace = True) 
cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,henderson,36.04,-114.98,38.89,12,1,7.70,US,1595536297
1,cherskiy,68.75,161.30,19.31,62,5,3.90,RU,1595536297
2,port alfred,-33.59,26.89,17.74,45,0,4.54,ZA,1595536028
3,santiago del estero,-27.80,-64.26,13.02,39,100,6.20,AR,1595536297
4,khatanga,71.97,102.50,5.79,90,100,7.44,RU,1595536297


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Confgure the map using humidity as the weight
gmaps.configure(api_key=g_key)
locations = cities_data[["Lat", "Lng"]]
humidity = cities_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add the layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Only pick the cities that have temperature between 21 and 26 degrees celsius, low wind speed, no clouds
cities_data = cities_data.loc[(cities_data['Max Temp'] >= 21) & (cities_data['Max Temp'] <= 27) & \
                              (cities_data['Wind Speed'] < 10) & (cities_data['Cloudiness'] ==0) & \
                              (cities_data['Humidity'] < 66)]
cities_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,healdsburg,38.61,-122.87,26.11,53,0,0.45,US,1595536301
71,sabha,27.04,14.43,26.74,21,0,3.69,LY,1595536323
91,san quintin,30.48,-115.95,21.15,62,0,5.42,MX,1595536147
125,cassilandia,-19.11,-51.73,25.53,37,0,2.55,BR,1595536295
148,arraial do cabo,-22.97,-42.03,27.00,47,0,4.60,BR,1595536166
189,naples,40.83,14.25,25.56,65,0,1.00,IT,1595536189
216,morondava,-20.28,44.28,22.73,50,0,8.01,MG,1595536374
244,lata,40.16,-8.33,23.33,65,0,2.68,PT,1595536392
295,guiratinga,-16.35,-53.76,26.77,29,0,1.47,BR,1595536412
316,lompoc,34.64,-120.46,22.00,56,0,2.10,US,1595536410


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a space in the data frame for the nearest hotel
cities_data["Hotel Name"] = ""
cities_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,healdsburg,38.61,-122.87,26.11,53,0,0.45,US,1595536301,
71,sabha,27.04,14.43,26.74,21,0,3.69,LY,1595536323,
91,san quintin,30.48,-115.95,21.15,62,0,5.42,MX,1595536147,
125,cassilandia,-19.11,-51.73,25.53,37,0,2.55,BR,1595536295,
148,arraial do cabo,-22.97,-42.03,27.00,47,0,4.60,BR,1595536166,
189,naples,40.83,14.25,25.56,65,0,1.00,IT,1595536189,
216,morondava,-20.28,44.28,22.73,50,0,8.01,MG,1595536374,
244,lata,40.16,-8.33,23.33,65,0,2.68,PT,1595536392,
295,guiratinga,-16.35,-53.76,26.77,29,0,1.47,BR,1595536412,
316,lompoc,34.64,-120.46,22.00,56,0,2.10,US,1595536410,


In [6]:
# Set up the parameters need for google's near by search
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Go through the rows in the data frame
for index, row in cities_data.iterrows():
  
    # Get the longitude and latitude and add to params
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # Start building the url for the api
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Reguest the data
    hotels = requests.get(base_url, params=params)
    hotels = hotels.json()
    
    # Try to get a hotels, if one is found, increase counter
    try:
        cities_data.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        pass

In [7]:
# Create a dataframe with the 10 cities that have hotels
cities_10_data = cities_data.loc[cities_data['Hotel Name'] != '']
cities_10_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,healdsburg,38.61,-122.87,26.11,53,0,0.45,US,1595536301,Harmon Guest House
71,sabha,27.04,14.43,26.74,21,0,3.69,LY,1595536323,فندق المواجيد
91,san quintin,30.48,-115.95,21.15,62,0,5.42,MX,1595536147,Old Mill Hotel
125,cassilandia,-19.11,-51.73,25.53,37,0,2.55,BR,1595536295,Hotel Rafa Cassilândia
148,arraial do cabo,-22.97,-42.03,27.00,47,0,4.60,BR,1595536166,Orlanova Hotel
189,naples,40.83,14.25,25.56,65,0,1.00,IT,1595536189,Renaissance Naples Hotel Mediterraneo
216,morondava,-20.28,44.28,22.73,50,0,8.01,MG,1595536374,Kimony Resort Hotel
295,guiratinga,-16.35,-53.76,26.77,29,0,1.47,BR,1595536412,Ópera Palace Hotel
316,lompoc,34.64,-120.46,22.00,56,0,2.10,US,1595536410,Hilton Garden Inn Lompoc
317,pisco,-13.70,-76.22,21.11,64,0,7.20,PE,1595536423,Hotel Campestre Corina


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_10_data.iterrows()]
locations = cities_10_data[["Lat", "Lng"]]

In [9]:
# Create the heatmap again with the humity as weight and the 10 cities we have
humidity = cities_10_data["Humidity"].astype(float)

gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Add the markers for the 10 cities
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))